# **Laboratorio 6**
## **Parte 2**

- Derek Arreaga - 22537
- Paula Barillas - 22764
- Mónica Salvatierra - 22249

## **Teoría** | **Parte 2**

Defina y responda

**1. La matriz Jacobiana es una "instantánea" de las fuerzas del sistema en un punto específico. Explique conceptualmente qué representa el elemento en la esquina inferior izquierda de la matriz (∂(dI/dt)/∂S). ¿Qué nos dice este término sobre la relación entre los susceptibles y la propagación de la infección?**

- El término (∂(dI/dt)/∂S) representa cómo va cambiando la velocidad de crecimiento de los infectados (dI/dt) cuando se modifica ligeramente la cantidad de personas de susceptibles (S), manteniendo todo lo demás constante. Es decir, indica cuánto influye el número de personas susceptibles en la propagación de la infección. Si este valor es positivo y grande, significa que un pequeño aumento en los susceptibles genera un incremento notable en el número de infecciones, mostrando que el sistema es muy sensible a los cambios en la población susceptible.

<br>

**2. El análisis muestra que cuando R₀ > 1, el Equilibrio Libre de Enfermedad es inestable. Describa en una o dos frases qué significa esto en el contexto de una política de salud pública de "esperar y ver". ¿Por qué la inestabilidad de este punto hace que esa política sea arriesgada?**

- Cuando R₀ > 1, el equilibrio libre de enfermedad es inestable, lo que significa que si aparece un pequeño número de infectados, la enfermedad tenderá a propagarse en lugar de desaparecer. En una política de salud pública de “esperar y ver”, esto implica un riesgo alto, porque un brote inicial mínimo puede ocasionar que los contagios crezcan rápidamente. Por eso, no intervenir desde el inicio puede permitir que la epidemia se descontrole antes de poder reaccionar.

<br>


**3. El Equilibrio Endémico, cuando existe, suele ser estable. ¿Qué significa "estabilidad" en este contexto? Si una nueva variante del virus, más contagiosa, causa un aumento repentino de casos (una perturbación), ¿qué predice la estabilidad de este equilibrio sobre el comportamiento del sistema a largo plazo?**

- La estabilidad del equilibrio endémico significa que, si el sistema se desvía ligeramente de ese punto, tenderá a regresar gradualmente a su estado estable. Si surge una variante más contagiosa que provoque un pico repentino de casos, la estabilidad predice que, con el tiempo, los contagios volverán a un nivel constante, aunque ese nuevo nivel podría ser más alto dependiendo del valor actualizado de R₀.

<br>

**4. En un modelo de ecuaciones, un equilibrio estable es un punto fijo. En un MBA, debido a la aleatoriedad (estocasticidad), el sistema nunca se asienta perfectamente en un punto. ¿Cómo se manifestaría un "equilibrio estable" en el espacio de estados proyectado de un MBA? Describa cómo se vería la trayectoria del sistema una vez que ha alcanzado este estado de equilibrio.**

- En un modelo basado en agentes, la estabilidad no se ve como un punto fijo exacto, ya que la aleatoriedad hace que los valores varíen constantemente. Un equilibrio estable se manifestaría como una región densa en el espacio de estados, donde el sistema oscila alrededor de un promedio sin desviarse mucho. La trayectoria del sistema mostraría cambios pequeños y continuos alrededor de ese punto de equilibrio, en lugar de converger a un valor único.

<br>

## **Práctica** | **Parte 2**

### **Librerias**

In [7]:
import sympy as sp
import numpy as np

### **Definición simbólica del modelo y Jacobiano**

In [2]:

# Símbolos
t = sp.symbols('t', real=True)
S, I = sp.symbols('S I', real=True)
beta, gamma, mu, N = sp.symbols('beta gamma mu N', positive=True, real=True)

# Ecuaciones del modelo (sin R explícito)
dS_dt = mu*N - beta*S*I - mu*S
dI_dt = beta*S*I - gamma*I - mu*I

# Vectores
eqs = sp.Matrix([dS_dt, dI_dt])
vars = sp.Matrix([S, I])

# Jacobiano simbólico
J = eqs.jacobian(vars)
print("Jacobiano simbólico J(S,I):")
sp.pprint(J)

Jacobiano simbólico J(S,I):
⎡-I⋅β - μ     -S⋅β    ⎤
⎢                     ⎥
⎣  I⋅β     S⋅β - γ - μ⎦
⎡-I⋅β - μ     -S⋅β    ⎤
⎢                     ⎥
⎣  I⋅β     S⋅β - γ - μ⎦


### **Autovalores simbólicos en el ELE (S=N, I=0)**

In [8]:
J_ELE = J.subs({S: N, I: 0})
print("Jacobiano en ELE (S=N, I=0):")
sp.pprint(J_ELE)

# Autovalores simbólicos
eigs_ELE = J_ELE.eigenvals()
print("\nAutovalores simbólicos en ELE:")
for ev, mult in eigs_ELE.items():
    print(f"  λ = {sp.simplify(ev)}  (multiplicidad {mult})")

Jacobiano en ELE (S=N, I=0):
⎡-μ     -N⋅β    ⎤
⎢               ⎥
⎣0   N⋅β - γ - μ⎦

Autovalores simbólicos en ELE:
  λ = N*beta - gamma - mu  (multiplicidad 1)
  λ = -mu  (multiplicidad 1)


### **Parámetros numéricos y equilibrio endémico**

In [9]:
N_val = 1000
beta_val = 0.5 / N_val 
gamma_val = 0.1
mu_val = 0.02

# Coordenadas del equilibrio endémico (teóricas para SIR con vitales y beta normalizada por N)
S_end_val = (gamma_val + mu_val) / beta_val
I_end_val = mu_val * (N_val - S_end_val) / (gamma_val + mu_val)

print("Parámetros numéricos:")
print(dict(N=N_val, beta=beta_val, gamma=gamma_val, mu=mu_val))
print("Equilibrio endémico numérico (S_end, I_end):", (S_end_val, I_end_val))

Parámetros numéricos:
{'N': 1000, 'beta': 0.0005, 'gamma': 0.1, 'mu': 0.02}
Equilibrio endémico numérico (S_end, I_end): (240.0, 126.66666666666667)


### **Jacobiano numérico en el equilibrio endémico**

In [10]:
# Sustitución de parámetros primero y luego de S_end, I_end
params_subs = {beta: beta_val, gamma: gamma_val, mu: mu_val, N: N_val}
J_numeric_sym = J.subs(params_subs).subs({S: S_end_val, I: I_end_val})

print("Jacobiano numérico en el equilibrio endémico:")
sp.pprint(J_numeric_sym)

# Convertimos a numpy
J_numeric = np.array(J_numeric_sym, dtype=np.float64)
print("\nJacobiano numérico (numpy array):\n", J_numeric)

Jacobiano numérico en el equilibrio endémico:
⎡-0.0833333333333333          -0.12        ⎤
⎢                                          ⎥
⎣0.0633333333333333   -1.38777878078145e-17⎦

Jacobiano numérico (numpy array):
 [[-8.33333333e-02 -1.20000000e-01]
 [ 6.33333333e-02 -1.38777878e-17]]


### **Autovalores numéricos del Jacobiano endémico**

In [11]:
eigs_end = np.linalg.eigvals(J_numeric)
print("Autovalores numéricos en el equilibrio endémico:")
print(eigs_end)

Autovalores numéricos en el equilibrio endémico:
[-0.04166667+0.07657603j -0.04166667-0.07657603j]


### **Preguntas de Análisis**

**1. Observe el segundo autovalor simbólico que obtuvo para el ELE. Muestre algebraicamente cómo este autovalor es igual a (γ + μ) * (R₀ - 1). ¿Cómo confirma esto su hallazgo teórico de que la inestabilidad ocurre cuando R₀ > 1?**

Se observa que  el segundo autovalor del Equilibrio Libre de Enfermedad es $\lambda_2 = N * \beta - \gamma - \mu$. Para demostrar algebraicamente que este autovalor es igual a $(\gamma + \mu) * (R_0 - 1)$, en donde el número reproductivo básico se define como $R_0 = \frac{(\beta * N)}{\gamma + \mu}$, y se reescribe el autovalor mediante manipulación algebraica: $$\lambda_2 = N * \beta - (\gamma + \mu) = (\gamma + \mu) * [N * \frac{\beta}{\gamma + \mu} - 1] = (\gamma + \mu)*(R_0 - 1)$$

En donde esta expresión confirma la teoria, ya que cuando $R_0 > 1 \rightarrow (R_0 - 1) > 0$, y como $(\gamma + \mu)$ es siempre positivo por ser tasas de salida del compartimento de infectados el autovalor $\lambda_2$ es positivo.En donde las perturbaciones en esa dirección como la aparición de infectados, crecen exponencialmente en el tiempo teniendo como consecuencia que el ELE sea inestable y el sistema evolucione inevitablemente hacia el equilibrio endémico.

Con nuestros parámetros , tenemos $R_0 = 4.167$, por lo que $(R_0 - 1) = 3.167$ y $(\gamma + \mu) = 0.12$, resultando en $\lambda_2 = 0.12 * 3.167 = 0.38$. En donde este valor positivo confirma cuantitativamente que si introducimos un solo infectado en una población de 1000 susceptibles que se encuentra en el ELE; el número de infectados crece inicialmente a una tasa exponencial proporcional a $e^{(0.38*t)}$, de manera que después de apenas 5 días el crecimiento sería aproximadamente $e^{(0.38×5)}\approx 6.7$ veces el valor inicial, donde se demuestra  matemáticamente la inestabilidad del ELE y consideramos que se entiende el por qué las epidemias no se extinguen espontáneamente cuando $R_0 > 1$.

**2. Examine los autovalores numéricos que calculó para el equilibrio endémico. ¿Sus partes reales son positivas o negativas? Basado en estas reglas, ¿el equilibrio endémico es estable o inestable?**

En este caso los autovalores numéricos del equilibrio endémico que calculamos son $\lambda_1 \approx -0.0417 + 0.0766 i$ y $\lambda_2 \approx -0.0417 - 0.0766 i$. En donde se puede observar  que ambos tienen parte real negativa, en este caso  $Re(\lambda) ≈ -0.0417 < 0$. En donde según la teoría de estabilidad lineal, un equilibrio es estable si y solo si todos sus autovalores tienen parte real negativa, por lo tanto el equilibrio endémico es estable y esta actuando como un atractor en el sistema. La parte real negativa (-0.0417) indica que las perturbaciones decaen exponencialmente con una tasa aproximada de $e^{(-0.0417·t)}$, lo que significa que una perturbación inicial se reduce a la mitad en aproximadamente $t \approx \frac{ln(2)}{0.0417} \approx 16.6 \text{ días}$.

En el caso de la parte imaginaria ($\pm 0.0766$) indica oscilaciones con un periodo $T \approx \frac{2\pi}{0.0766} \approx 82 \text{días}$, por lo cuál si el sistema se perturba, por ejemplo mediante una vacunación masiva que reduce temporalmente el número de infectados; el sistema regresará al equilibrio de forma oscilatoria pero amortiguada con ciclos de aproximadamente 82 días que se van apagando gradualmente hasta estabilizarse en el punto de equilibrio endémico.

**3. Compare los resultados de su análisis de estabilidad (Parte 2) con el comportamiento visual de las trayectorias que graficó en la Parte 1. ¿Son consistentes los resultados? Explique cómo el hecho de que el ELE sea inestable y el Endémico sea estable explica por qué las trayectorias se comportan como lo hacen.**

En este caso los resultados son totalmente consistentes En las simulaciones del plano de fases observamos que ambas trayectorias (azul y morada) convergen al punto verde $X$ ubicado en $(S*, I*) \approx (240, 126.67)$, mientras que ninguna trayectoria se acerca al punto cian que representa el Equilibrio Libre de Enfermedad en (1000, 0). Ya que la trayectoria azul muestra un gran arco inicial antes de estabilizarse, lo cual tiene una explicación matemática precisa. En donde el ELE es inestable porque su autovalor $\lambda_2 = +0.38 > 0$ lo convierte en un repulsor, lo que significa que cualquier introducción de infectados como por ejemplo I₀ = 1, hace que el sistema se aleje exponencialmente de este equilibrio.

Por otro lado, el equilibrio endémico es estable porque sus autovalores tienen parte real negativa $Re(\lambda) = -0.0417 < 0$, donde lo convierte en un atractor hacia el cual todas las trayectorias son jaladas. El gran arco que muestra la trayectoria azul que inicia en (999, 1) se explica porque inicialmente hay muchos susceptibles ($S \approx 999$) lo que genera contagios rápidos en la fase ascendente del arco, haciendo que $I$ crezca hasta alcanzar aproximadamente 450-460 infectados en el pico. En donde al agotarse los susceptibles efectivos I comienza a decrecer en la fase descendente del arco, y finalmente las oscilaciones amortiguadas causadas por la parte imaginaria de $\lambda$ hacen que la trayectoria forme una espiral convergente hacia el punto (240, 126.67). 

**4. La esperanza de vida en una población aumenta, lo que significa que μ disminuye. ¿Cómo afectaría una μ más pequeña al autovalor positivo del ELE cuando R₀ > 1? ¿Haría que el brote inicial fuera más rápido o más lento? Explique su razonamiento basándose en la fórmula del autovalor.**

Utilizando la fórmula directa $\lambda_2 = N·β - (\gamma + \mu)$, donde $N*\beta$ permanece constante. En el escenario actual con $\mu = 0.02$, el cuál corresponde a una esperanza de vida de $\frac{1}{0.02} = 50$ unidades de tiempo, tenemos $\lambda_2 = 1000×0.0005 - (0.1 + 0.02) = 0.5 - 0.12 = 0.38$. Por lo cual si consideramos un escenario con mayor esperanza de vida donde μ se reduce a la mitad (sería $\mu_{nueva} = 0.01$, equivalente a una esperanza de vida de 100 unidades), entonces $\lambda_{2 nueva} = 1000×0.0005 - (0.1 + 0.01) = 0.5 - 0.11 = 0.39$.

Se observa que $\lambda_2$ aumenta de **0.38** a **0.39** cuando $\mu$ disminuye, lo que significa que el brote inicial sería más rápido. Ya que esto se debe a que un $\lambda_2$ mayor implica una tasa exponencial de crecimiento más alta para los infectados cerca del ELE, siguiendo la relación $I(t) \approx I_0·e^{(λ₂·t)}$. Por ejemplo con $\mu = 0.02$ tenemos $I(5 días) \approx I_0·e^{(0.38*5)} \approx 6.7*I_0$, mientras que con $\mu = 0.01$ tendríamos $I(5 \text{días}) \approx I_0·e^{(0.39×5)} \approx 7.0·I_0$, representando aproximadamente **4.5%** más infectados en el mismo periodo. En donde este resultado puede parecer contraintuitivo, pero consideremos que tiene sentido epidemiológico ya que una población con menor $\mu$ hace que la epidemia se propague más rápido en su fase inicial porque en teoría hay menos salidas del compartimento de infectados por muerte natural, lo que permite que los individuos susceptibles e infectados permanezcan más tiempo en la población aumentando las oportunidades de transmisión. En donde esta dinámica hace más urgente la intervención temprana en poblaciones con mayor esperanza de vida, ya que esperar incluso unos pocos días puede resultar en miles de casos adicionales debido al mayor valor de $\lambda_2$.

## **Prompt Utilizado**

Uso de ChatGPT

Hola otra vez, estoyy trabajando en análisis de estabilidad de modelos epidemiológicos SIR con dinámica vital para un laboratorio de Modelación y Simulación. Necesito implementar en Python el cálculo de la matriz Jacobiana simbólica usando `sympy`, evaluar autovalores en los puntos de equilibrio (ELE y endémico), y luego convertir a formato numérico con `numpy` para análisis cuantitativo. El objetivo es determinar si los equilibrios son estables (atractores) o inestables (repulsores) basándome en el signo de las partes reales de los autovalores, y conectar estos resultados con las trayectorias simuladas previamente."

#### **Por qué funcionó este prompt**

El prompt funcionó porque fue específico sobre el contexto técnico (análisis de estabilidad mediante Jacobiano y autovalores) sin revelar la solución completa del problema. Mencionó las herramientas exactas necesarias (`sympy` para cálculo simbólico, `numpy` para conversión numérica), el flujo de trabajo (simbólico → evaluación en equilibrios → numérico), y el criterio de estabilidad (signo de partes reales). Esto permitió que la herramienta generara código modular y reutilizable para calcular `.jacobian()`, usar `.subs()` con diccionarios de parámetros, evaluar `.eigenvals()`, y convertir con `np.array().astype(np.float64)`, todo mientras mantenía la conexión conceptual entre el análisis matemático y las simulaciones previas del espacio de estados.

### **Referencias**

1. Equilibrio endémico. (s. f.). ScienceDirect. Recuperado 16 de octubre de 2025, de https://www.sciencedirect.com/topics/mathematics/endemic-equilibrium

2. Zhang, X., Wang, J., Yu, C., Fei, J., Luo, T., & Cao, Z. (2025). Agent-Based Modeling of Epidemics: Approaches, applications, and future directions. Technologies, 13(7), 272. https://doi.org/10.3390/technologies13070272
